# Barplot als Aktivitätsdiagramm

In [64]:
import itertools
import locale

from IPython.display import display, Markdown, HTML

locale.setlocale(locale.LC_ALL, "de_DE.utf8")

def create_bars(specs, max_actions=10000, scaling=lambda x: x, descaling=lambda x: x,
                grey="#ddd", height_per_bar=100, text_height=25, bins=4):
    
    def create_hline(y, color, progress=1):
        return """<line y1="%s" x1="5" y2="%s" x2="%s" stroke="%s"
                        stroke-linecap="round"
                        stroke-width="10"/>""" % (y, y, progress*1000+5, color)
    
    result = []
    
    height = height_per_bar * len(specs) + text_height
    
    result.append('<svg style="width: 100%%" viewBox="0 0 1050 %s">' % height)
        
    for i in range(bins):
        x = 1 + 1000/bins * (i+1)
        number = int(descaling((i+1)/bins * scaling(max_actions)))
        text = "{:,}".format(number)
        
        result.append("""
            <text x="%s" y="%s" text-anchor="middle"
                  style="font-size: 16px;">%s</text>
        """ % (x, text_height-10, text))
        result.append("""
            <line y1="%s" x1="%s" y2="%s" x2="%s" stroke="#111"
             stroke-width="2" stroke-dasharray="10"/>
        """ % (text_height, x, height, x))
    
    for spec, y in zip(specs, itertools.count(height_per_bar/2 + text_height, height_per_bar)):
        progress = min(1, scaling(spec["actions"]) / scaling(max_actions))
        
        result.append(create_hline(y, grey))
        result.append(create_hline(y, spec["color"], progress=progress))
        
        label = "%s %s" % (spec["actions"], spec["kind"])
        result.append("""
            <text x="0" y="%s" style="font-weight: bold; font-size: 18px;">%s</text>
        """ % (y+40, label))
        
    result.append('</svg>')
    
    return "\n".join(result)

specs = [
    {"color": "#1E65A7", "actions": 8078, "kind": "Bearbeitungen"},
    {"color": "#F1CD04", "actions": 567, "kind": "Kommentare"},
    {"color": "#00743F", "actions": 2498, "kind": "Reviewing"},
    {"color": "#D87A00", "actions": 123, "kind": "Themenbaum"},
]

display(Markdown("## Lineare Skala"))
display(HTML(create_bars(specs)))

## Lineare Skala

In [67]:
import math

display(Markdown("## Exponentielle Skala"))
display(HTML(create_bars(specs, scaling=math.log10, descaling=lambda x: 10**x)))

## Exponentielle Skala

In [68]:
import math

display(Markdown("## Potenzierte Skala"))
display(HTML(create_bars(specs, scaling=math.sqrt, descaling=lambda x: x**2)))

## Potenzierte Skala

In [99]:
def polarToCartesian(centerX, centerY, radius, angleInDegrees):
        angleInRadians = angleInDegrees * math.pi / 180.0

        return ( centerX + (radius * math.cos(angleInRadians)),
                 centerY + (radius * math.sin(angleInRadians)))
    
def circle(x, y, radius, color, progress=1, width=1, startAngle=275):
    endAngle = startAngle + 360*progress - 1

    start = polarToCartesian(x, y, radius, endAngle)
    end = polarToCartesian(x, y, radius, startAngle)

    largeArcFlag = "0" if endAngle - startAngle <= 180 else "1"

    d = " ".join(map(str, [
        "M", start[0], start[1], 
        "A", radius, radius, 0, largeArcFlag, 0, end[0], end[1]
    ]))

    return """
        <path d="%s" fill="none" stroke-linecap="round"
              stroke="%s" stroke-width="%s" />
    """ % (d, color, width)

def create_level_circle(actions, color="#1E65A7", max_actions=10000, scaling=lambda x: x/100):
    progress = scaling(min(max_actions, actions))
    level = math.floor(progress)
    progress = progress - level
    
    result = []
    
    result.append('<svg viewBox="0 0 100 100" style="width: 100%;">')
    result.append(circle(50, 50, 40, color, progress=progress, width=5, startAngle=45))
    
    cx, cy = polarToCartesian(50, 50, 40, 45)
    
    result.append("""
        <circle cx="%s" cy="%s" r="10" fill="white" />
    """ % (cx, cy))
    result.append("""
        <circle cx="%s" cy="%s" r="8" fill="%s" stroke="#e1bf00"
                stroke-width="1"/>
    """ % (cx, cy, "#f1cd04"))
    result.append("""
        <text x="%s" y="%s" text-anchor="middle"
              style="font-size: 8px;">%s</text>
    """ % (cx, cy+3, level))
    
    result.append('</svg>')
    
    return "\n".join(result)
    
display(HTML(create_level_circle(1290)))

In [105]:
def create_level_diagram(specs, scaling=lambda x: x/100):
    result = []
    
    result.append('<div style="width: 100%; display: flex; flex-flow: row wrap; justify-content: space-around;">')
    for spec in specs:
        result.append('<div style="width: 25%; min-width: 200px">')
        result.append(create_level_circle(spec["actions"], spec["color"], scaling=scaling))
        result.append('<p><b>%s %s</b></p>' % (spec["actions"], spec["kind"]))
        result.append('</div>')
    
    result.append("</div>")
        
    return "\n".join(result)    

display(Markdown("## Lineare Skala"))
display(HTML(create_level_diagram(specs)))

## Lineare Skala

In [107]:
display(Markdown("## Wurzel-Skala"))
display(HTML(create_level_diagram(specs, scaling=math.sqrt)))

## Wurzel-Skala

In [110]:
display(Markdown("## Exponentielle Skala"))
display(HTML(create_level_diagram(specs, scaling=math.log10 )))

## Exponentielle Skala

In [112]:
display(Markdown("## Exponentielle Skala (mit 100 Levels)"))
display(HTML(create_level_diagram(specs, scaling=lambda x: math.log(x) / math.log(10000**0.01) )))

## Exponentielle Skala (mit 100 Levels)